# Copy and paste of the notebook's lab on FFNN w/ Addition of a Kaf Layer
* Taken from the course: "Neural Networks for Data Science Applications a.a. 19-20 Sapienza Univerisy of Rome"
* Accuracy improvements gained thanks to kafs
* Note, this notebook requires the SUSY dataset in the working folder (https://archive.ics.uci.edu/ml/datasets/SUSY)

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from kaf import Kaf # local source


In [2]:
# Sanity check 

#x = tf.random.uniform(shape=(3,28,28,3), minval=-20, maxval=40, dtype=tf.float32)
x = tf.random.uniform(shape=(3,10), minval=-20, maxval=40, dtype=tf.float32)

#kaf = Kaf(5, conv=True)
kaf = Kaf(5)
kaf(x)


<tf.Tensor: shape=(3, 10), dtype=float32, numpy=
array([[-4.2984564e-02,  3.7445154e-02, -7.9597086e-02,  2.4790630e-02,
        -5.1283002e-02,  6.0842138e-02, -4.2399783e-02,  2.4390133e-02,
        -1.4805279e-05,  8.4657788e-02],
       [ 2.4514697e-03,  6.1873361e-02, -8.4410638e-02,  1.1055615e-02,
        -8.2367964e-02,  6.0414318e-02,  6.7392655e-04,  3.5295114e-03,
         2.3100324e-02,  2.7735978e-02],
       [-4.4142682e-02,  3.9227179e-04, -3.2701310e-02,  3.0161624e-04,
        -7.0682101e-02,  1.9684648e-03,  4.4078249e-03,  1.6381934e-02,
        -1.1540288e-02,  1.8637761e-03]], dtype=float32)>

In [3]:
# Import SUSY Dataset used during the lab of NNfDS

import pandas as pd

susy = pd.read_csv('SUSY.csv', header=None)
X = susy.values[0:100000, 1:].astype(np.float32) # Note the casting to float32
y = susy.values[0:100000, 0:1]

train_susy = tf.data.Dataset.from_tensor_slices((X, y))

In [4]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential

def build_model():
  net = Sequential()
  net.add(Dense(50))
  net.add(Kaf(30))
  net.add(Dense(1, activation='sigmoid'))
  return net

In [5]:
net = build_model()

In [6]:
from tensorflow.keras import losses, optimizers, metrics

net.compile(optimizer=optimizers.Adam(), loss=losses.BinaryCrossentropy(), 
           metrics = [
               metrics.BinaryAccuracy(),
               metrics.AUC()
           ])

In [7]:
history = net.fit(train_susy.shuffle(1000).batch(64), epochs=3)

Train for 1563 steps
Epoch 1/3
1563/1563 [==============================] - 12s 8ms/step - loss: 0.4739 - binary_accuracy: 0.7762 - auc: 0.8457
Epoch 2/3
1563/1563 [==============================] - 13s 8ms/step - loss: 0.4438 - binary_accuracy: 0.7946 - auc: 0.8653
Epoch 3/3
1563/1563 [==============================] - 13s 8ms/step - loss: 0.4407 - binary_accuracy: 0.7958 - auc: 0.8671


In [8]:
net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  950       
_________________________________________________________________
kaf_1 (Kaf)                  multiple                  1500      
_________________________________________________________________
dense_1 (Dense)              multiple                  51        
Total params: 2,501
Trainable params: 2,501
Non-trainable params: 0
_________________________________________________________________


In [9]:
# We use a separate set of 100k values
Xtest = susy.values[100000:200000, 1:].astype(np.float32)
ytest = susy.values[100000:200000, 0:1]

In [10]:
test_susy = tf.data.Dataset.from_tensor_slices((Xtest, ytest))

In [11]:
# We remove the last dimension from y
test_susy = tf.data.Dataset.from_tensor_slices((Xtest, 
                                                ytest.reshape(-1)))

In [12]:
# Better score than during the lab session!
net.evaluate(test_susy.batch(32))

3125/3125 [==============================] - 16s 5ms/step - loss: 0.4356 - binary_accuracy: 0.7997 - auc: 0.8711


[0.4355582052183151, 0.79975, 0.8710929]